In [1]:
import pandas as pd
import torch as tc
import numpy as np
import multiprocessing
import re
from transformers import DistilBertTokenizerFast
import torch
from transformers import DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
# from pyevmasm import disassemble_hex


/home/venglov/.venv/main/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# benign_df = pd.read_csv('../data/normal_df.csv').sample(frac=1).reset_index(drop=True)[:20000]
# malicious_df = pd.read_csv('../data/malicious.csv')
# sc_df = pd.concat([benign_df, malicious_df]).reset_index(drop=True)

In [2]:
sc_df = pd.read_csv('/home/venglov/Documents/mcd_data/data_v2/opcodes_full_10k.csv')

In [3]:
sc_df.head()

,Unnamed: 0,contract_address,opcodes,malicious
0,0,0x0016394e785f667bb472d4148ab88a5c391accdf,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False
1,1,0xbdc4dd4b2f62c446485a1d2e39569466a2fbd829,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,False
2,2,0x3e9ba4be6affa324f188b10962317239e32ce7ad,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x00 PUSH1 ...,False
3,3,0xf5194c3325202f456c95c1cf0ca36f8475c1949f,PUSH1 0x60 PUSH2 0x375a PUSH2 0x0140 CODECOPY ...,False
4,4,0x5869d838be4f1efa10f2082efc957c79f7c3bf0b,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH3 0x00001b PU...,False


In [6]:
# sc_df = sc_df[:10000]

In [5]:
# sc_df = sc_df[['decompiled_opcodes', 'malicious']].sample(frac=1).reset_index(drop=True)

In [6]:
# sc_df = sc_df.drop(sc_df[sc_df.decompiled_code == "error"].index)
# sc_df = sc_df.drop(sc_df[sc_df.decompiled_code == "error ;"].index)
# sc_df = sc_df.reset_index()
# sc_df

In [4]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


In [8]:
# sc_df.opcodes.tolist()[0]

In [5]:
inputs = tokenizer(sc_df.opcodes.tolist(), padding="max_length", truncation=True)

In [6]:
class SmartContractsDataset(tc.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # self.sc_df = sc_df
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # code = self.sc_df.decompiled_code[idx]
        # code = simple_web3_code_decompile(code)
        # code = preprocess_code(code)
        # code = self.get_contract_as_image(code).transpose(2, 0, 1)
        # code = tc.tensor(code).type(tc.float32)
        # code = tokenizer(code).items()

        # mal = self.sc_df.malicious[idx]
        # mal = tc.tensor(mal).type(tc.float32)
        # return code, mal

        # item = {key: torch.tensor(val[idx]) for key, val in code}
        # item['label'] = mal
        # return item
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item


In [7]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [8]:
device = "cuda"


In [9]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [16]:
scd = SmartContractsDataset(inputs, sc_df.malicious.tolist())
train_size = int(0.8 * len(scd))
val_size = int(0.1 * len(scd))
test_size = len(scd) - train_size - val_size
training_dataset, validation_dataset, testing_dataset = tc.utils.data.random_split(scd,
                                                                                   [train_size, val_size, test_size])


In [17]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    # use_mps_device=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
)

In [ ]:
trainer.train()

/home/venglov/.venv/main/lib64/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=5060, training_loss=0.011367063457261323, metrics={'train_runtime': 891.3113, 'train_samples_per_second': 90.788, 'train_steps_per_second': 5.677, 'total_flos': 1.071926189924352e+16, 'train_loss': 0.011367063457261323, 'epoch': 10.0})

In [19]:
results = []
for i in range(len(testing_dataset)):
    code = testing_dataset[i]
    label = code['label']
    input_ids = torch.tensor(code["input_ids"]).to(device)
    attention_mask = torch.tensor(code["attention_mask"]).to(device)
    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
    y = np.argmax(outputs[0].to('cpu').numpy())
    results.append((label, y))

/tmp/ipykernel_5330/1724829149.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(code["input_ids"]).to(device)
/tmp/ipykernel_5330/1724829149.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(code["attention_mask"]).to(device)


In [20]:
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0

# Generates count of inference quadrants
for real, predicted in results:
    if real == 1 and predicted == 1:
        true_positive += 1
    elif real == 0 and predicted == 0:
        true_negative += 1
    elif real == 1 and predicted == 0:
        false_negative += 1
    elif real == 0 and predicted == 1:
        false_positive += 1

# print(l2)
print("True Positive:", true_positive)
print("True Negative:", true_negative)
print("False Positive:", false_positive)
print("False Negative:", false_negative)

# Machine Learning statistics and visuals. https://towardsdatascience.com/accuracy-recall-precision-f-score-specificity-which-to-optimize-on-867d3f11124
print("Accuracy:", (true_positive + true_negative)/(true_positive + true_negative + false_positive + false_negative))
precision = true_positive/(true_positive + false_positive)
print("Precision:", precision)
recall = true_positive/(true_positive + false_negative)
print("Recall:", recall)
print("F1-score", 2*(recall * precision)/(recall + precision))
print("Specificity:", true_negative/(true_negative + false_positive))

True Positive: 12
True Negative: 1000
False Positive: 0
False Negative: 1
Accuracy: 0.9990128331688055
Precision: 1.0
Recall: 0.9230769230769231
F1-score 0.9600000000000001
Specificity: 1.0


[(tensor(0), 0),
 (tensor(1), 1),
 (tensor(0), 0),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(0), 0),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(0), 0),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(0), 0),
 (tensor(0), 0),
 (tensor(1), 1)]

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
real = []
predicted = []
for r, p in results:
    real.append(r)
    predicted.append(p)
cf_matrix = confusion_matrix(real, predicted)
df_cm = pd.DataFrame(cf_matrix, index=[0, 1],
                     columns=[0, 1])
plt.figure(figsize=(12, 7))
sn.heatmap(df_cm, annot=True)

In [21]:
torch.save(model.state_dict(), '/home/venglov/Documents/mcd_data/data_v2/opcodes_model_weights_v2.pth')

In [22]:
model.load_state_dict(torch.load("/home/venglov/Documents/mcd_data/data_v2/opcodes_model_weights_v2.pth"))

<All keys matched successfully>

In [ ]:
address = "0x792e8f3727cad6e00c58d478798f0907c4cec340"


In [23]:
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [24]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [25]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


In [ ]:
web3 = Web3(Web3.HTTPProvider("https://eth-mainnet.gateway.pokt.network/v1/lb/6266d6cdaa777e00391e0d29"))
code = web3.eth.getCode(Web3.toChecksumAddress(address))

In [26]:
code = "0x6080604052600436106100585760003560e01c8062f714ce146100b657806371a70b6e146100df5780638da5cb5b1461011c57806397feb926146101475780639db5dbe414610170578063b69ef8a814610199576100b1565b366100b157346001600082825461006f9190610668565b925050819055507ff1b03f708b9c39f453fe3f0cef84164c7d6f7df836df0796e1e9c2bce6ee397e33346040516100a79291906106ec565b60405180910390a1005b600080fd5b3480156100c257600080fd5b506100dd60048036038101906100d89190610793565b6101c4565b005b3480156100eb57600080fd5b5061010660048036038101906101019190610945565b610336565b60405161011391906109a1565b60405180910390f35b34801561012857600080fd5b50610131610371565b60405161013e91906109bc565b60405180910390f35b34801561015357600080fd5b5061016e60048036038101906101699190610a15565b610395565b005b34801561017c57600080fd5b5061019760048036038101906101929190610a55565b61041a565b005b3480156101a557600080fd5b506101ae610629565b6040516101bb91906109a1565b60405180910390f35b60008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614610252576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161024990610b05565b60405180910390fd5b600154821115610297576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161028e90610b71565b60405180910390fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc839081150290604051600060405180830381858888f193505050501580156102dd573d6000803e3d6000fd5b5081600160008282546102f09190610b91565b925050819055507ffda3a3e0e1479b43cb1c701f7576187f4c4ad80768d627387e00184302f7d88e33828460405161032a93929190610c24565b60405180910390a15050565b600260205281600052604060002081805160208101820180518482526020830160208501208183528095505050505050600091509150505481565b60008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b8173ffffffffffffffffffffffffffffffffffffffff166323b872dd3330846040518463ffffffff1660e01b81526004016103d293929190610c5b565b6020604051808303816000875af11580156103f1573d6000803e3d6000fd5b505050506040513d601f19601f820116820180604052508101906104159190610cca565b505050565b60008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146104a8576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161049f90610b05565b60405180910390fd5b60008373ffffffffffffffffffffffffffffffffffffffff166370a08231306040518263ffffffff1660e01b81526004016104e391906109bc565b602060405180830381865afa158015610500573d6000803e3d6000fd5b505050506040513d601f19601f820116820180604052508101906105249190610d0c565b905080821115610569576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161056090610d85565b60405180910390fd5b8373ffffffffffffffffffffffffffffffffffffffff1663a9059cbb84846040518363ffffffff1660e01b81526004016105a49291906106ec565b6020604051808303816000875af11580156105c3573d6000803e3d6000fd5b505050506040513d601f19601f820116820180604052508101906105e79190610cca565b507ffda3a3e0e1479b43cb1c701f7576187f4c4ad80768d627387e00184302f7d88e33848460405161061b93929190610c5b565b60405180910390a150505050565b60015481565b6000819050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052601160045260246000fd5b60006106738261062f565b915061067e8361062f565b925082820190508082111561069657610695610639565b5b92915050565b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b60006106c78261069c565b9050919050565b6106d7816106bc565b82525050565b6106e68161062f565b82525050565b600060408201905061070160008301856106ce565b61070e60208301846106dd565b9392505050565b6000604051905090565b600080fd5b600080fd5b6107328161062f565b811461073d57600080fd5b50565b60008135905061074f81610729565b92915050565b60006107608261069c565b9050919050565b61077081610755565b811461077b57600080fd5b50565b60008135905061078d81610767565b92915050565b600080604083850312156107aa576107a961071f565b5b60006107b885828601610740565b92505060206107c98582860161077e565b9150509250929050565b6107dc816106bc565b81146107e757600080fd5b50565b6000813590506107f9816107d3565b92915050565b600080fd5b600080fd5b6000601f19601f8301169050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052604160045260246000fd5b61085282610809565b810181811067ffffffffffffffff821117156108715761087061081a565b5b80604052505050565b6000610884610715565b90506108908282610849565b919050565b600067ffffffffffffffff8211156108b0576108af61081a565b5b6108b982610809565b9050602081019050919050565b82818337600083830152505050565b60006108e86108e384610895565b61087a565b90508281526020810184848401111561090457610903610804565b5b61090f8482856108c6565b509392505050565b600082601f83011261092c5761092b6107ff565b5b813561093c8482602086016108d5565b91505092915050565b6000806040838503121561095c5761095b61071f565b5b600061096a858286016107ea565b925050602083013567ffffffffffffffff81111561098b5761098a610724565b5b61099785828601610917565b9150509250929050565b60006020820190506109b660008301846106dd565b92915050565b60006020820190506109d160008301846106ce565b92915050565b60006109e2826106bc565b9050919050565b6109f2816109d7565b81146109fd57600080fd5b50565b600081359050610a0f816109e9565b92915050565b60008060408385031215610a2c57610a2b61071f565b5b6000610a3a85828601610a00565b9250506020610a4b85828601610740565b9150509250929050565b600080600060608486031215610a6e57610a6d61071f565b5b6000610a7c86828701610a00565b9350506020610a8d868287016107ea565b9250506040610a9e86828701610740565b9150509250925092565b600082825260208201905092915050565b7f4f6e6c79206f776e65722063616e2077697468647261772066756e6473000000600082015250565b6000610aef601d83610aa8565b9150610afa82610ab9565b602082019050919050565b60006020820190508181036000830152610b1e81610ae2565b9050919050565b7f496e73756666696369656e742066756e64730000000000000000000000000000600082015250565b6000610b5b601283610aa8565b9150610b6682610b25565b602082019050919050565b60006020820190508181036000830152610b8a81610b4e565b9050919050565b6000610b9c8261062f565b9150610ba78361062f565b9250828203905081811115610bbf57610bbe610639565b5b92915050565b6000819050919050565b6000610bea610be5610be08461069c565b610bc5565b61069c565b9050919050565b6000610bfc82610bcf565b9050919050565b6000610c0e82610bf1565b9050919050565b610c1e81610c03565b82525050565b6000606082019050610c3960008301866106ce565b610c466020830185610c15565b610c5360408301846106dd565b949350505050565b6000606082019050610c7060008301866106ce565b610c7d60208301856106ce565b610c8a60408301846106dd565b949350505050565b60008115159050919050565b610ca781610c92565b8114610cb257600080fd5b50565b600081519050610cc481610c9e565b92915050565b600060208284031215610ce057610cdf61071f565b5b6000610cee84828501610cb5565b91505092915050565b600081519050610d0681610729565b92915050565b600060208284031215610d2257610d2161071f565b5b6000610d3084828501610cf7565b91505092915050565b7f62616c616e6365206973206c6f77000000000000000000000000000000000000600082015250565b6000610d6f600e83610aa8565b9150610d7a82610d39565b602082019050919050565b60006020820190508181036000830152610d9e81610d62565b905091905056fea2646970667358221220f48fd51c7ea6c59fa7a676bf6cfba0e98ba92beb29121c80030756ad91683a0f64736f6c63430008120033"

In [20]:
opcodes = disassemble_hex(code)


In [23]:
opcodes = opcodes.replace('\n', ' ')

AttributeError: 'list' object has no attribute 'join'

In [43]:
inputs = tokenizer([opcodes], padding="max_length", truncation=True)


In [44]:
inputs[2].items()

IndexError: list index out of range

In [45]:
item = {key: torch.tensor(val[0]) for key, val in inputs.items()}

In [47]:
item['input_ids']

tensor([  101,  5245,  2487,  1014,  2595, 17914,  5245,  2487,  1014,  2595,
        12740,  5796, 19277,  5245,  2487,  1014,  2595,  2549,  2655,  2850,
        10230,  4697,  8318,  5245,  2475,  1014,  2595, 27814,  5376,  2072,
         5245,  2487,  1014,  2595,  2692,  2655,  2850,  9080, 10441,  2094,
         5245,  2487,  1014,  2595,  2063,  2692, 14021,  2099,  4241,  2361,
         2487,  5245,  2509,  1014,  2595,  2546,  2581, 16932,  3401,  1041,
         4160,  5245,  2475,  1014,  2595,  2497,  2575,  5376,  2072,  4241,
         2361,  2487,  5245,  2549,  1014,  2595,  2581,  2487,  2050, 19841,
         2497,  2575,  2063,  1041,  4160,  5245,  2475,  1014,  2595, 20952,
         5376,  2072,  4241,  2361,  2487,  5245,  2549,  1014,  2595,  2620,
         2850,  2629, 27421,  2629,  2497,  1041,  4160,  5245,  2475,  1014,
         2595, 14526,  2278,  5376,  2072,  4241,  2361,  2487,  5245,  2549,
         1014,  2595,  2683,  2581,  7959,  2497,  2683, 23833, 

In [55]:
input_ids = torch.tensor(item["input_ids"]).to('cpu')
attention_mask = torch.tensor(item["attention_mask"]).to('cpu')
with torch.no_grad():
    outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
y = np.argmax(outputs[0].to('cpu').numpy())

/var/folders/tf/97gp74_92k30dq5fwdw7j5bm0000gn/T/ipykernel_23062/1113231631.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(item["input_ids"]).to('cpu')
/var/folders/tf/97gp74_92k30dq5fwdw7j5bm0000gn/T/ipykernel_23062/1113231631.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(item["attention_mask"]).to('cpu')


In [56]:
outputs[0]

tensor([[-2.1050,  2.4778]])

In [54]:
y

1